## Import libraries

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import json
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Job search Webcrawler

In [2]:
## sleep
def sleep_func(time_sec):
    print ('wating'),
    interval = 1
    c = 0
    while c < time_sec:
        time.sleep(interval)
        print ('.'),
        c += 1

In [3]:
def dict_append(dictionary, idx_counter, new_key, old_key1, old_key2, old_key3=None):
# add hiring organisation to ad dictionary
    if old_key3 == None:
        dictionary[idx_counter][new_key] = dictionary[idx_counter][old_key1][old_key2]
        print('.'),
        return dictionary
    else:
        if old_key3 in dictionary[idx_counter][old_key1][old_key2].keys():
            dictionary[idx_counter][new_key] = dictionary[idx_counter][old_key1][old_key2][old_key3]
            print('.'),
            return dictionary

In [102]:
def seek_webcrawler(cities, jobs, pages):
    
    start_time = time.time()
    driver = webdriver.Chrome('/home/vicki/Desktop/chromedriver') 
    
    ads_dict = {}
    idx_counter = 0
    exception = 0
    exception_list = []
    # repeat search for each city
    for city in cities:
        print ('starting city {}'.format(city)),
        # repeat search for each job type
        for job in jobs:
            print ('for {} jobs'.format(job)),
            # iterate search result pages
            for pgs in range(1,pages+1):
                print ('searching page {}\n\n'.format(pgs)),
                
                # make page request: Seek address format: https://www.seek.com/__jobtitle__+'jobs/'in-'+__city-name__'?page=''__#__
                #delete_all_cookies()
                url = ('https://www.seek.com.au/'+str(job)+'-jobs'+'/in-'+str(city)+'?page='+str(pgs))
                headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"}
                page = requests.get(url=url, headers=headers)
###########     
# https://www.quora.com/Web-Crawling-How-do-websites-tell-if-their-site-is-being-accessed-by-a-crawler-or-an-actual-user
                
                print ('{} {}\n'.format(page, url))

                #parse html information to BS
                soup = BeautifulSoup(page.text, 'html.parser')
                print ('.'),
                
                # Pull ad data from script tag for each page # add to dictionary
                for script in soup.find_all(name='script', attrs={'type':'application/ld+json'}):
                    
                    print ('.'),
                    
                    # Get the text of each script tag
                    script_text = script.text
                    # Decode json format
                    ad_data = json.loads(script_text)
                    # add json dict to ads_dict using counter as index
                    if 'legalName' not in ad_data.keys() and 'mainEntityOfPage' not in ad_data.keys(): 
                        ads_dict[idx_counter] = (ad_data)

                        # add hiring organisation to ad dictionary
                        dict_append(ads_dict, idx_counter, 'employer', 'hiringOrganization', 'name') 

                        # add city to ad dictionary
                        dict_append(ads_dict, idx_counter, 'city', 'jobLocation', 'address', 'addressRegion') 

                        # add locality to ad dictionary
                        dict_append(ads_dict, idx_counter, 'locality', 'jobLocation', 'address', 'addressLocality') 
                        
                        # add job title search to dictionary
                        ads_dict[idx_counter]['job_search_key'] = job
                        
                        # add job description to ad dictionary
                        driver.get(ads_dict[idx_counter]['url'])

                        try:
                            if driver.find_element_by_class_name('templatetext') is None:
                                print ('no description')
                            else:
                                description = driver.find_element_by_class_name('templatetext')
                                ads_dict[idx_counter]['description'] = description.text
                                print('.'),
                        except:
                            print ('exception @ site ---> {}'.format(ads_dict[idx_counter]['url'])),
                            exception += 1
                            exception_list.append(ads_dict[idx_counter]['url'])

                        if idx_counter !=0:    
                            if idx_counter % 50 == 0:
                                sleep_func(60)
                                print ('\n')
                            if idx_counter % 10 == 0:
                                sleep_func(10)
                                print ('\n')
                        ## add 1 to counter
                        idx_counter += 1

                        print (' Completed search {} for {} in {}\n'.format(idx_counter, job, city))
                        driver.delete_all_cookies()
                    else:
                        print 'seek info header'
    
    webdriver.Chrome.close
    print ('\nYour search is complete -- {} results returned'.format(idx_counter))
    if len(exception_list)>0:
        print ('There were {} exeptions @:\n'.format(exception, exception_list))
    print ('\nElapsed time: {} seconds'.format(time.time()-start_time))
    return (ads_dict)

## Web crawler conditions

In [103]:
## Set scrapping conditions 

cities = ['All-Melbourne-VIC', 'All-Sydney-NSW', 'All-Brisbane-QLD', 'All-Adelaide-SA','All-Perth-WA']
jobs = ['data-scientist', 'data-analyst', 'business-intelligence', 'business-analyst', 'data-architect',
       'data-engineer', 'statistician', 'research-scientist', 'analyst'] #etc

# 20 listings per page
pages = 5

## Run Webcrawler

In [105]:
ads_dict = seek_webcrawler(cities, jobs, pages)

starting city All-Melbourne-VIC for data-scientist jobs searching page 1

<Response [200]> https://www.seek.com.au/data-scientist-jobs/in-All-Melbourne-VIC?page=1

. . seek info header
. seek info header
. . . . .  Completed search 1 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 2 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 3 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 4 for data-scientist in All-Melbourne-VIC

. . . .  Completed search 5 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 6 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 7 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 8 for data-scientist in All-Melbourne-VIC

. . . .  Completed search 9 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 10 for data-scientist in All-Melbourne-VIC

. . . . . wating . . . . . . . . . . 

 Completed search 11 for data-scientist in All-Mel

<Response [200]> https://www.seek.com.au/data-analyst-jobs/in-All-Melbourne-VIC?page=3

. . seek info header
. seek info header
. . . . .  Completed search 96 for data-analyst in All-Melbourne-VIC

. . . . .  Completed search 97 for data-analyst in All-Melbourne-VIC

. . . .  Completed search 98 for data-analyst in All-Melbourne-VIC

. . . . .  Completed search 99 for data-analyst in All-Melbourne-VIC

. . . . .  Completed search 100 for data-analyst in All-Melbourne-VIC

. . . . . wating . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

wating . . . . . . . . . . 

 Completed search 101 for data-analyst in All-Melbourne-VIC

. . . . .  Completed search 102 for data-analyst in All-Melbourne-VIC

. . . . .  Completed search 103 for data-analyst in All-Melbourne-VIC

. . . . .  Completed search 104 for data-analyst in All-Melbourne-VIC

. . . . .  Completed search 105 for data-analyst in All-Melbourne-VIC

. . . . . 

. . . . .  Completed search 276 for business-analyst in All-Melbourne-VIC

. . . .  Completed search 277 for business-analyst in All-Melbourne-VIC

searching page 2

<Response [200]> https://www.seek.com.au/business-analyst-jobs/in-All-Melbourne-VIC?page=2

. . seek info header
. seek info header
. . . . .  Completed search 278 for business-analyst in All-Melbourne-VIC

. . . . .  Completed search 279 for business-analyst in All-Melbourne-VIC

. . . . .  Completed search 280 for business-analyst in All-Melbourne-VIC

. . . . . wating . . . . . . . . . . 

 Completed search 281 for business-analyst in All-Melbourne-VIC

. . . . .  Completed search 282 for business-analyst in All-Melbourne-VIC

. . . .  Completed search 283 for business-analyst in All-Melbourne-VIC

. . . . .  Completed search 284 for business-analyst in All-Melbourne-VIC

. . . . .  Completed search 285 for business-analyst in All-Melbourne-VIC

. . . . .  Completed search 286 for business-analyst in All-Melbourne-VIC



. . . . .  Completed search 369 for data-architect in All-Melbourne-VIC

. . . . .  Completed search 370 for data-architect in All-Melbourne-VIC

. . . . . wating . . . . . . . . . . 

 Completed search 371 for data-architect in All-Melbourne-VIC

. . . . .  Completed search 372 for data-architect in All-Melbourne-VIC

. . . . .  Completed search 373 for data-architect in All-Melbourne-VIC

. . . . .  Completed search 374 for data-architect in All-Melbourne-VIC

. . . . .  Completed search 375 for data-architect in All-Melbourne-VIC

. . . . .  Completed search 376 for data-architect in All-Melbourne-VIC

. . . . .  Completed search 377 for data-architect in All-Melbourne-VIC

. . . . .  Completed search 378 for data-architect in All-Melbourne-VIC

. . . .  Completed search 379 for data-architect in All-Melbourne-VIC

. . . . .  Completed search 380 for data-architect in All-Melbourne-VIC

. . . . wating . . . . . . . . . . 

 Completed search 381 for data-architect in All-Melbourne-VI

. . . . . wating . . . . . . . . . . 

 Completed search 461 for data-engineer in All-Melbourne-VIC

. . . . .  Completed search 462 for data-engineer in All-Melbourne-VIC

. . . . .  Completed search 463 for data-engineer in All-Melbourne-VIC

. . . .  Completed search 464 for data-engineer in All-Melbourne-VIC

. . . . .  Completed search 465 for data-engineer in All-Melbourne-VIC

. . . . .  Completed search 466 for data-engineer in All-Melbourne-VIC

. . . .  Completed search 467 for data-engineer in All-Melbourne-VIC

. . . .  Completed search 468 for data-engineer in All-Melbourne-VIC

. . . . .  Completed search 469 for data-engineer in All-Melbourne-VIC

. . . . .  Completed search 470 for data-engineer in All-Melbourne-VIC

. . . . wating . . . . . . . . . . 

 Completed search 471 for data-engineer in All-Melbourne-VIC

. . . .  Completed search 472 for data-engineer in All-Melbourne-VIC

. . . .  Completed search 473 for data-engineer in All-Melbourne-VIC

. . . .  Completed

...

<Response [200]> https://www.seek.com.au/statistician-jobs/in-All-Adelaide-SA?page=1

. . seek info header
. seek info header
. . . .  Completed search 1806 for statistician in All-Adelaide-SA

. . . .  Completed search 1807 for statistician in All-Adelaide-SA

. . . .  Completed search 1808 for statistician in All-Adelaide-SA

searching page 2

<Response [200]> https://www.seek.com.au/statistician-jobs/in-All-Adelaide-SA?page=2

. . seek info header
. seek info header
searching page 3

<Response [200]> https://www.seek.com.au/statistician-jobs/in-All-Adelaide-SA?page=3

. . seek info header
. seek info header
searching page 4

<Response [200]> https://www.seek.com.au/statistician-jobs/in-All-Adelaide-SA?page=4

. . seek info header
. seek info header
searching page 5

<Response [200]> https://www.seek.com.au/statistician-jobs/in-All-Adelaide-SA?page=5

. . seek info header
. seek info header
for research-scientist jobs searching page 1

<Response [200]> https://www.seek.com.au/researc

. . . .  Completed search 1896 for analyst in All-Adelaide-SA

. . . .  Completed search 1897 for analyst in All-Adelaide-SA

. . . .  Completed search 1898 for analyst in All-Adelaide-SA

. . . .  Completed search 1899 for analyst in All-Adelaide-SA

. . . .  Completed search 1900 for analyst in All-Adelaide-SA

. . . . wating . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

wating . . . . . . . . . . 

 Completed search 1901 for analyst in All-Adelaide-SA

. . . .  Completed search 1902 for analyst in All-Adelaide-SA

. . . .  Completed search 1903 for analyst in All-Adelaide-SA

. . . .  Completed search 1904 for analyst in All-Adelaide-SA

. . . .  Completed search 1905 for analyst in All-Adelaide-SA

. . . .  Completed search 1906 for analyst in All-Adelaide-SA

. . . .  Completed search 1907 for analyst in All-Adelaide-SA

. . . .  Completed search 1908 for analyst in All-Adelaide-SA

. . . .  Completed sear

. . . .  Completed search 1972 for business-analyst in All-Perth-WA

. . . . .  Completed search 1973 for business-analyst in All-Perth-WA

. . . . .  Completed search 1974 for business-analyst in All-Perth-WA

. . . .  Completed search 1975 for business-analyst in All-Perth-WA

. . . . .  Completed search 1976 for business-analyst in All-Perth-WA

. . . . .  Completed search 1977 for business-analyst in All-Perth-WA

. . . .  Completed search 1978 for business-analyst in All-Perth-WA

. . . . .  Completed search 1979 for business-analyst in All-Perth-WA

. . . . .  Completed search 1980 for business-analyst in All-Perth-WA

. . . . . wating . . . . . . . . . . 

 Completed search 1981 for business-analyst in All-Perth-WA

. . . . .  Completed search 1982 for business-analyst in All-Perth-WA

. . . . .  Completed search 1983 for business-analyst in All-Perth-WA

. . . . .  Completed search 1984 for business-analyst in All-Perth-WA

. . . .  Completed search 1985 for business-analyst in

. . . . wating . . . . . . . . . . 

 Completed search 2071 for data-architect in All-Perth-WA

. . . . .  Completed search 2072 for data-architect in All-Perth-WA

. . . .  Completed search 2073 for data-architect in All-Perth-WA

. . . .  Completed search 2074 for data-architect in All-Perth-WA

. . . . .  Completed search 2075 for data-architect in All-Perth-WA

. . . . .  Completed search 2076 for data-architect in All-Perth-WA

searching page 2

<Response [200]> https://www.seek.com.au/data-architect-jobs/in-All-Perth-WA?page=2

. . seek info header
. seek info header
searching page 3

<Response [200]> https://www.seek.com.au/data-architect-jobs/in-All-Perth-WA?page=3

. . seek info header
. seek info header
searching page 4

<Response [200]> https://www.seek.com.au/data-architect-jobs/in-All-Perth-WA?page=4

. . seek info header
. seek info header
searching page 5

<Response [200]> https://www.seek.com.au/data-architect-jobs/in-All-Perth-WA?page=5

. . seek info header
. seek inf

. . . . .  Completed search 2144 for analyst in All-Perth-WA

. . . .  Completed search 2145 for analyst in All-Perth-WA

. . . . .  Completed search 2146 for analyst in All-Perth-WA

. . . . .  Completed search 2147 for analyst in All-Perth-WA

. . . . .  Completed search 2148 for analyst in All-Perth-WA

. . . . .  Completed search 2149 for analyst in All-Perth-WA

. . . . .  Completed search 2150 for analyst in All-Perth-WA

. . . . wating . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

wating . . . . . . . . . . 

 Completed search 2151 for analyst in All-Perth-WA

. . . . .  Completed search 2152 for analyst in All-Perth-WA

. . . . .  Completed search 2153 for analyst in All-Perth-WA

. . . . .  Completed search 2154 for analyst in All-Perth-WA

. . . . .  Completed search 2155 for analyst in All-Perth-WA

searching page 4

<Response [200]> https://www.seek.com.au/analyst-jobs/in-All-Perth-WA?page=4

. . se

In [106]:
ads_dict[2]['job_search_key']

'data-scientist'

## Make dataframe

In [107]:
jobs_df = pd.DataFrame(ads_dict)
jobs_df = jobs_df.T

## Save data to csv

In [108]:
jobs_df.to_csv('seek_jobs_raw_17_Sept.csv', encoding='utf-8')

## Format dataframe 

In [109]:
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2195 entries, 0 to 2194
Data columns (total 14 columns):
@context              2195 non-null object
@type                 2195 non-null object
city                  2195 non-null object
datePosted            2195 non-null object
description           2195 non-null object
employer              2195 non-null object
employmentType        2195 non-null object
hiringOrganization    2195 non-null object
industry              2195 non-null object
jobLocation           2195 non-null object
job_search_key        2195 non-null object
locality              1647 non-null object
title                 2195 non-null object
url                   2195 non-null object
dtypes: object(14)
memory usage: 257.2+ KB


In [110]:
jobs_df.head(1)

,@context,@type,city,datePosted,description,employer,employmentType,hiringOrganization,industry,jobLocation,job_search_key,locality,title,url
0,http://schema.org,JobPosting,Melbourne,2017-09-16T14:00:00Z,About us\nCrypton Analytics is a high performi...,Crypton Analytics,Full Time,"{u'@type': u'Organization', u'name': u'Crypton...",Science & Technology,"{u'@type': u'Place', u'address': {u'addressLoc...",data-scientist,CBD & Inner Suburbs,Data Scientist (Cryptocurrency),https://www.seek.com.au/job/34374593


In [111]:
jobs_df.describe().iloc[1:2,:]

,@context,@type,city,datePosted,description,employer,employmentType,hiringOrganization,industry,jobLocation,job_search_key,locality,title,url
unique,1,1,5,1626,1559,684,4,684,27,27,9,19,1188,1742


In [112]:
jobs_df = jobs_df.rename(columns={'datePosted':'date_posted','employmentType':'type'})
jobs_df.industry.unique()

array([u'Science & Technology', u'Information & Communication Technology',
       u'Banking & Financial Services', u'Consulting & Strategy',
       u'Education & Training', u'Mining, Resources & Energy',
       u'Manufacturing, Transport & Logistics',
       u'Administration & Office Support', u'Government & Defence',
       u'Call Centre & Customer Service', u'Sales', u'Accounting',
       u'Marketing & Communications', u'Retail & Consumer Products',
       u'Engineering', u'Trades & Services',
       u'Human Resources & Recruitment', u'Advertising, Arts & Media',
       u'Healthcare & Medical', u'Design & Architecture', u'Construction',
       u'Insurance & Superannuation', u'Legal', u'Real Estate & Property',
       u'CEO & General Management', u'Hospitality & Tourism',
       u'Community Services & Development'], dtype=object)

### search for duplicate listings

In [113]:
# search for duplicate listings
#jobs_df[jobs_df.duplicated(['description'], keep = 'first')]
jobs_clean = jobs_df.drop_duplicates(subset='description', keep='first')

In [114]:
jobs_df.shape, jobs_clean.shape, jobs_df[jobs_df.duplicated(['description'], keep = 'first')].shape

((2195, 14), (1559, 14), (636, 14))

In [115]:
jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1559 entries, 0 to 2192
Data columns (total 14 columns):
@context              1559 non-null object
@type                 1559 non-null object
city                  1559 non-null object
date_posted           1559 non-null object
description           1559 non-null object
employer              1559 non-null object
type                  1559 non-null object
hiringOrganization    1559 non-null object
industry              1559 non-null object
jobLocation           1559 non-null object
job_search_key        1559 non-null object
locality              1206 non-null object
title                 1559 non-null object
url                   1559 non-null object
dtypes: object(14)
memory usage: 182.7+ KB


In [116]:
jobs_clean.describe().iloc[1:2,:]

,@context,@type,city,date_posted,description,employer,type,hiringOrganization,industry,jobLocation,job_search_key,locality,title,url
unique,1,1,5,1508,1559,684,4,684,27,27,9,19,1176,1554


### Drop extra columns

In [117]:
jobs_clean = jobs_clean.drop(['@context','@type','jobLocation','hiringOrganization'], axis = 1)

## Save clean dataframe to csv

In [118]:
jobs_clean.to_csv('seek_jobs_clean_17_Sept.csv', encoding='utf-8')

---------------------------------